In [9]:

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from random import choice


"""
=============================================================================
 VAE-GMM Stock Prediction Model
-----------------------------------------------------------------------------
 This file implements an end-to-end pipeline for:
   1. Data Ingestion & Preprocessing
   2. Variational Autoencoder (VAE) Training
   3. Gaussian Mixture Model (GMM) Clustering
   4. Probability Prediction & Buy/Sell Signal Generation
   5. Backtesting
   6. (Optional) Real-time Inference Pipeline

 Reference: VAE-GMM Stock Prediction Model Specification Document
=============================================================================
"""


 ======================= Section 1: Data Ingestion ===========================
# Ref. Specification 2.1: "Data Ingestion: The system should collect OHLCV 
# price data, fundamentals, and macroeconomic indicators."


In [10]:

class StockDataset(Dataset):
    """
    A custom PyTorch dataset to handle stock time series and fundamental data.
    Reference: Specification 3.1 - Data Sources & 3.2 - Data Preprocessing
    """

    def __init__(self, period, transform):

        self.period = period #day/ csvs(minute)
        self.dfs = {}
        self.transform = transform
        
        
        for name in ('IWM', 'NVDA', 'QQQ', 'SPY', 'TSLA'):

            self.file = f"{self.period}/{name}.csv"
            if not os.path.exists(self.file):  # ✅ Check if file exists before loading
                print(f"⚠️ Warning: File {self.file} not found. Skipping...")
                continue  # Skip this stock if file doesn't exist


            # load
            df = pd.read_csv(self.file, parse_dates=['Datetime'], index_col='Datetime')
            df.index = pd.to_datetime(df.index, errors='coerce')
            # Basic cleaning of missing values (forward fill)
            df.ffill()
            self.dfs.update({name : df})

    def __len__(self):
        return sum(len(df) for df in dfs)

    def __getitem__(self, key_idx):
        # Example features: [Open, High, Low, Close, Volume]
        stock, idx = key_idx
        if stock not in self.dfs:
            raise KeyError(f"Stock '{stock}' not found in dataset.")

        df = self.dfs[stock]  # Retrieve the correct DataFrame

        if idx >= len(df):
            raise IndexError(f"Index {idx} out of bounds for '{stock}'.")
    
        row = df.iloc[idx]
        features = np.array([row["Open"], row["High"], row["Low"],
                             row["Close"], row["Volume"]], dtype=np.float32)
        
        # (Optional) apply transformations if needed
        if self.transform:
            features = self.transform(features)

        return features



# ================== Section 2: Data Preprocessing & Utilities ===============
# Ref. Specification 3.2: "Data Preprocessing Steps: Normalize numerical data, 
# log-scale, handle missing, etc."


In [11]:

def preprocess_data(dataset):
    """
    Normalize and preprocess a dictionary-based dataset for training a VAE:
    - Normalizes numerical data per stock
    - Converts to a single NumPy array for batch training
    - Stores individual scalers per stock
    
    Returns:
        - `processed_data` (np.ndarray): Concatenated dataset (all stocks)
        - `scaler_dict` (dict): Dictionary of scalers for each stock
        - `stock_labels` (np.ndarray): Stock identifiers for each row
    """
    scaler_dict = {}  # Stores scalers per stock
    all_features = []  # Stores all preprocessed features
    stock_labels = []  # Stock identifiers

    for stock_symbol, df in dataset.dfs.items():
        if df.empty:
            continue  # Skip empty DataFrames

        # Convert DataFrame to NumPy array
        features = df[["Open", "High", "Low", "Close", "Volume"]].values

        # Normalize using StandardScaler per stock
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(features)

        # Append to all_features list
        all_features.append(scaled_features)

        # Keep track of which stock each row belongs to
        stock_labels.append(np.full(len(scaled_features), stock_symbol))

        # Store the scaler for inverse transform later
        scaler_dict[stock_symbol] = scaler

    # Stack all stock data into a single NumPy array
    processed_data = np.vstack(all_features)  # Shape: (total_rows, 5)
    stock_labels = np.concatenate(stock_labels)  # Shape: (total_rows,)

    return processed_data, scaler_dict, stock_labels



# ===================== Section 3: Variational Autoencoder ====================
# Ref. Specification 4.1: "VAE Model: Train a Variational Autoencoder (VAE)
# to extract latent features."

In [12]:
class VAE(nn.Module):
    """
    A simple Variational Autoencoder for stock time series as described in
    Specification 4.1.
    """

    def __init__(self, input_dim=5, latent_dim=2):
        """
        Args:
            input_dim (int): Number of input features (e.g., OHLCV).
            latent_dim (int): Dimensionality of the latent space.
        """
        super(VAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2_mu = nn.Linear(16, latent_dim)
        self.fc2_logvar = nn.Linear(16, latent_dim)

        # Decoder
        self.fc3 = nn.Linear(latent_dim, 16)
        self.fc4 = nn.Linear(16, input_dim)

    def encode(self, x):
        h = torch.relu(self.fc1(x))
        mu = self.fc2_mu(h)
        logvar = self.fc2_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = torch.relu(self.fc3(z))
        return self.fc4(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar


def vae_loss_function(reconstructed, original, mu, logvar):
    """
    Combination of Reconstruction Loss (MSE) and KL Divergence.
    Reference: Specification 4.1.
    """
    mse = nn.MSELoss(reduction="sum")
    recon_loss = mse(reconstructed, original)

    # KL Divergence
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kld

def train_vae(model, data, epochs=10, batch_size=64, lr=1e-3):
    """
    Train a Variational Autoencoder (VAE) using the preprocessed stock dataset.

    Args:
        model (VAE): The VAE model.
        data (np.ndarray): Preprocessed numerical stock data.
        epochs (int): Number of training epochs.
        batch_size (int): Batch size.
        lr (float): Learning rate.
    """
    # Convert data to PyTorch tensor
    dataset = TensorDataset(torch.tensor(data, dtype=torch.float32))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for batch_features in dataloader:
            batch_features = batch_features[0]  # Extract from dataset tuple
            
            optimizer.zero_grad()
            reconstructed, mu, logvar = model(batch_features)

            loss = vae_loss_function(reconstructed, batch_features, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")



# ==================== Section 4: Gaussian Mixture Model ======================
# Ref. Specification 4.2: "GMM Model: Train a Gaussian Mixture Model (GMM) 
# to cluster the latent space into different market regimes."

In [13]:

def train_gmm(latent_features, n_components=3):
    """
    Train a Gaussian Mixture Model on the latent features.
    
    Args:
        latent_features (np.ndarray): Latent representations from the VAE.
        n_components (int): Number of mixture components.
        
    Returns:
        gmm (GaussianMixture): Trained GMM model.
    """
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(latent_features)
    return gmm


# ============= Section 5: Buy/Sell Decision Mechanism & Backtesting ==========
# Ref. Specification 4.3: "If Bullish Probability > 70% -> BUY, etc."

In [14]:

def generate_signals(probabilities, bullish_index=0, threshold=0.7):
    """
    Generate buy/sell/hold signals based on probability distribution from GMM.

    Args:
        probabilities (np.ndarray): Probabilities for each GMM component.
        bullish_index (int): Index of the GMM component considered "bullish".
        threshold (float): Probability threshold for buy/sell signals.

    Returns:
        signals (list): List of signals ("BUY", "SELL", or "HOLD").
    """
    signals = []
    for prob in probabilities:
        # Example: If cluster '0' is bullish, cluster '1' might be bearish, etc.
        bullish_prob = prob[bullish_index]  # e.g., first cluster as bullish
        if bullish_prob > threshold:
            signals.append("BUY")
        elif bullish_prob < (1 - threshold):
            signals.append("SELL")
        else:
            signals.append("HOLD")
    return signals


def backtest(signals, price_data, index_dates, ticker, tax_rate=0.25, trading_fee=5.0):
    """
    Simulated backtesting function with correct tax on capital gains/losses.
    
    - Tax is applied only on the realized profit (Sell Price - Buy Price).
    - Losses generate a negative tax credit, which offsets future gains.
    - Trading fees are deducted on both buy and sell transactions.
    
    Args:
        signals (list): List of "BUY"/"SELL" signals.
        price_data (np.array): Corresponding stock prices.
        index_dates (pd.Index): Date index for backtest period.
        tax_rate (float): Tax rate applied on capital gains.
        trading_fee (float): Flat fee per trade.
    
    Returns:
        dict: Performance metrics including tax adjustments and portfolio value.
    """

    initial_cash = 10000
    cash = initial_cash
    holdings = 0
    total_fees = 0
    total_taxes = 0
    tax_credit = 0  # Track losses to offset future gains
    cost_basis = []  # Store purchase prices for accurate profit calculation

    # Ensure valid index_dates
    index_dates = pd.Series(index_dates).dropna().reset_index(drop=True)

    if index_dates.empty:
        print("⚠️ Warning: No valid dates available for backtest.")
        return None  # Exit if no valid dates

    max_idx = min(len(signals), len(price_data), len(index_dates))

    for i in range(max_idx):
        price = float(price_data[i])  # Ensure price is a float

        if signals[i] == "BUY" and cash > (price + trading_fee):
            holdings += 1
            cash -= (price + trading_fee)
            total_fees += trading_fee
            cost_basis.append(price)  # Store buy price

        elif signals[i] == "SELL" and holdings > 0:
            buy_price = cost_basis.pop(0)  # Get first buy price (FIFO method)
            profit = price - buy_price  # Capital gain/loss

            # Calculate tax only on profit
            tax = profit * tax_rate

            # If loss, store negative tax as credit
            if tax < 0:
                tax_credit += abs(tax)  # Store as a positive offset for future gains
                tax = 0  # No tax charged on losses
            else:
                if tax_credit > 0:
                    offset = min(tax, tax_credit)  # Use available credit
                    tax -= offset
                    tax_credit -= offset

            cash += (price - tax - trading_fee)  # Add net sale amount
            holdings -= 1
            total_fees += trading_fee
            total_taxes += tax

    # Calculate final portfolio value
    final_value = cash + (holdings * price_data[-1])  # Use last price for remaining stocks

    # Extract backtest period
    start_date = index_dates.iloc[0] if len(index_dates) > 0 else "N/A"
    end_date = index_dates.iloc[-1] if len(index_dates) > 0 else "N/A"

    # Calculate time delta safely
    time_delta = (end_date - start_date).days if isinstance(start_date, pd.Timestamp) and isinstance(end_date, pd.Timestamp) else "N/A"

    final_value = round(cash + (holdings * price_data[-1]), 2)
    revanue = (final_value - initial_cash) / time_delta


    return {
        "Ticker" : ticker,
        "Final Portfolio Value": final_value,
        "Cash Remaining": round(cash, 2),
        "Holdings": holdings,
        "Total Fees Paid": round(total_fees, 2),
        "Total Taxes Paid": round(total_taxes, 2),
        "Backtest Period": f"{start_date} → {end_date}",
        "Start Date": start_date,
        "End Date": end_date,
        "Time Delta (Days)": time_delta,
        "yearly upside" : revanue*365
    }





# ==================== Section 6: Putting It All Together =====================

In [15]:
def main():
    """
    Main entry point for training and inference pipeline.
    Reference: Specification 6.1 & 6.2 for Deployment and Real-time integration.
    """

    # ------------------------- Step 1: Data Ingestion ------------------------
    tickers = ['IWM', 'NVDA', 'QQQ', 'SPY', 'TSLA']  
    dataset = StockDataset('day', transform=True)

    # --------------------- Step 2: Data Preprocessing -----------------------
    scaled_data, scaler, labels = preprocess_data(dataset)

    # -------------------- Step 3: Train the VAE for Latent Features ---------
    vae = VAE(input_dim=5, latent_dim=2)
    train_vae(model=vae, data=scaled_data, epochs=5, batch_size=32, lr=1e-3)

    # Obtain latent features
    vae.eval()
    with torch.no_grad():
        latent_list = []
        for row in scaled_data:
            row_torch = torch.tensor(row, dtype=torch.float32).unsqueeze(0)
            mu, logvar = vae.encode(row_torch)
            z = vae.reparameterize(mu, logvar)
            latent_list.append(z.numpy().squeeze())
    latent_features = np.array(latent_list)

    # --------------------- Step 4: Train the GMM on Latent ------------------
    gmm = train_gmm(latent_features, n_components=3)

    # Example assumption: cluster 0 is bullish, 1 is bearish, 2 is neutral
    probabilities = gmm.predict_proba(latent_features)
    signals = generate_signals(probabilities, bullish_index=0, threshold=0.7)

    # ------------------------ Step 5: Backtesting ---------------------------
    # For demonstration, we’ll just use the Close price from the dataset
    # Note: Ensure indices match properly with your data alignment
    # randomaize testing
    ticker = choice(tickers)
    min_length = min(len(signals), len(dataset.dfs[ticker]["Close"].values))
    price_data = dataset.dfs[ticker]["Close"].values[:min_length]
    signals = signals[:min_length]  # Trim signals to match price_data length
    index_dates = index_dates = dataset.dfs[ticker].index[:min_length]  # Extract corresponding dates


    performance = backtest(signals, price_data, index_dates, ticker)
    print("Backtesting Performance:")
    for k, v in performance.items():
        print(f"{k}: {v}")

    # --------------------- Step 6: Real-time / Next Steps -------------------
    # Integration with a live pipeline would require:
    #   1. Fetching new data in real-time
    #   2. Preprocessing with the same scaler
    #   3. Getting latent features from the VAE
    #   4. Predicting cluster probabilities via GMM
    #   5. Generating signals for each new data point
    print("\nReal-time integration would involve repeating these steps with latest data.")


In [16]:
if __name__ == "__main__":
    main()


Epoch [1/5], Loss: 85498.3376
Epoch [2/5], Loss: 63142.3455
Epoch [3/5], Loss: 61824.3766
Epoch [4/5], Loss: 60982.3312
Epoch [5/5], Loss: 60639.7539
Backtesting Performance:
Ticker: QQQ
Final Portfolio Value: 83108.14
Cash Remaining: 1066.22
Holdings: 157
Total Fees Paid: 2885.0
Total Taxes Paid: 143.5
Backtest Period: 1999-03-10 00:00:00-05:00 → 2024-12-27 00:00:00-05:00
Start Date: 1999-03-10 00:00:00-05:00
End Date: 2024-12-27 00:00:00-05:00
Time Delta (Days): 9424
yearly upside: 2831.544047113752

Real-time integration would involve repeating these steps with latest data.
